# Implementing a machine learning for Titanic case

*For data anlysis see Titanic notebook*
##### First trials
* Load Input Data
  * Clean & complete data
  * Split Training Data 
* Train 1 ML 
* Validate using Input Data

##### Several ML implementing
First three steps kepts
* Train several ML
* Validate and compare
* Select Best
* Predict
* Submit

In [95]:
import pandas as pd
import numpy as np
import math

# Load Input Data

In [69]:
def LoadData(path):
    return pd.read_csv(path)
wholeInputData = LoadData('train.csv')
wholeInputData.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


## Clean & complete data

### Number of familly members
We add the number of familly member. It is dependent of two others variables but it wouldn't it be complicated for the ml to find this feature ?

In [70]:
def AddFamillyNbr(data):
    data['FamillyNbr']=data.Parch+data.SibSp
AddFamillyNbr(wholeInputData)

### Missing Age values
We fill the missing value for the age

In [80]:
def FillWithMedian(data, columnName):
    median = data[columnName].median()
    data[columnName]=data[columnName].fillna(median)
    
FillWithMedian(wholeInputData,'Age')

### Is There other empty values

In [73]:
def CheckForNull(ser):
    return ser.isnull().any()
print('Does Fare has NA : %s' % CheckForNull(wholeInputData.Fare))
print('Does Sex has NA : %s' % CheckForNull(wholeInputData.Sex))
print('Does Clas has NA : %s' % CheckForNull(wholeInputData.Pclass))
print('Does FamillyNbr has NA : %s' % CheckForNull(wholeInputData.FamillyNbr))
print('Does Ticket has NA : %s' % CheckForNull(wholeInputData.Ticket))
print('Does Cabin has NA : %s' % CheckForNull(wholeInputData.Cabin))

Does Fare has NA : False
Does Sex has NA : False
Does Clas has NA : False
Does FamillyNbr has NA : False
Does Ticket has NA : False
Does Cabin has NA : True


#### Cabin
Cabin is not useful as it is partially empty and has unique values.
Lets analyze the data to see if the First letter could be computed. 

In [109]:
wholeInputData[['Pclass','Cabin']].head(30)

,Pclass,Cabin
0,3,NaN
1,1,C85
2,3,NaN
3,1,C123
4,3,NaN
5,3,NaN
6,1,E46
7,3,NaN
8,3,NaN
9,2,NaN


Data won't be usable

### Split Training Data

In [62]:
def SplitData(input, sizeValidate):
    validateSample = input.sample(n=sizeValidate, random_state =10)#to be able to repeat if necessary we force randomstate
    rest = input.drop(validateSample.index)    
    return (validateSample, rest)
    
validateData, trainData=SplitData(wholeInputData,50)
len(validateData.index)

50

In [63]:
len(trainData.index)

841

#### Save them in case of

In [64]:
validateData.to_csv('split_validate.csv')
trainData.to_csv('split_train.csv')

## Train a decision tree
The decision tree is the easiest to understand. That's why I choosed it as the first

*This requires no missing value. No other preparation is needed*
#### Tidy Dataset

In [112]:
tidyValidateData = validateData.drop(['Cabin','Name','Ticket','Embarked'],axis=1)
tidyTrainData= trainData.drop(['Cabin','Name','Ticket','Embarked'],axis=1)